# MEWpy Simulation 

This notebook exemplifies how MEWpy may be used for phenotype simulation.

Models can be loaded using MEW, REFRAMED, or COBRApy. 

Load a model using REFRAMED: 

In [ ]:
! pip install mewpy --upgrade --quiet

In [ ]:
import mewpy
mewpy.info()

IMPORTANT: The notebooks require a MEWpy version >= 0.1.16

In [ ]:
from reframed.io.sbml import load_cbmodel
model = load_cbmodel('models/ec/e_coli_core.xml.gz')

or using COBRApy:

In [ ]:
from cobra.io import read_sbml_model
model = read_sbml_model('models/ec/e_coli_core.xml.gz')

A simulator object provides a common interface to realize the main phenotype analysis tasks. The *get_simulator* function returns a simulator, a wrapper,  for the provided model. The simulator interface remains the same regardless of how the model was loaded, using REFRAMED or COBRApy. This simplify the use of both environments and ease the management of future changes and deprecation on their APIs.


In [ ]:
from mewpy.simulation import get_simulator
simul = get_simulator(model)

The simulator offers a wide API, and enable to perform basic tasks, such as, list  metabolites, reactions, genes, compartments, uptake reactions, and transport reactions:

In [ ]:
# first 10 metabolites
simul.metabolites[:10]

In [ ]:
# first 10 metabolites
simul.reactions[:10]

In [ ]:
# first 10 genes
simul.genes[:10]

In [ ]:
simul.compartments

For a more compreensive look at the list of reactions, you may use the `find` function, without any parameters (list all reactions), or by adding a _filter_:

In [ ]:
simul.find()

In [ ]:
simul.find(['EX_','BIO'])

In [ ]:
# metabolites
simul.find_metabolites()

In [ ]:
# metabolites
simul.find_genes()

Listing the uptake reactions:

In [ ]:
simul.get_uptake_reactions()

In [ ]:
# for a more compreensive listage
simul.find(simul.get_uptake_reactions())

A simulator may also be loaded considering environmental conditions, that will be considered during phenotype simulations. In the next example, glucose consumption is limited to 10 mmol/gDW/h in anaerobical conditions.

In [ ]:
envcond = {'EX_glc__D_e': (-10.0, 100000.0),
           'EX_o2_e':(0,0)}

simul = get_simulator(model,envcond=envcond)

All phenotype simulations will consider the imposed environmental conditions, and as such they only need to be set once. Also, these conditions do not persistently alter the model, which can be reused with a different simulator instance.

## Phenotype simulation

Phenotype simulations are also run using the simulator instance using  the `simulate` method. 

In [ ]:
# FBA 
result = simul.simulate()
# or 
result = simul.simulate(method='FBA')

result

Flux Balance Analysis (FBA) can be run without identifying any method, or  by passing the 'FBA' as method parameter. Other phenotype simulation methods may also be run using one of the identifiers:

- Flux Balance Analysis: `method = 'FBA'`
- Parsimonious FBA:`method = 'pFBA'`
- Minimization of Metabolic Adjustment:`method = 'MOMA'`
- Linear MOMA: `method = 'lMOMA'`
- Regulatory on/off minimization of metabolic flux: `method = 'ROOM'`

In [ ]:
# pFBA
result = simul.simulate(method = 'pFBA')
result

## Reaction fluxes

The phenotype simulation result object, besides containing the objective value and solver status, also include reaction fluxes in the form of a dictionary:


In [ ]:
result.fluxes

or as a dataframe:

In [ ]:
result.find()

Individual reaction flux values can be obtained from the dictionary representation. For example, the *triose-phosphate isomerase* (TPI) reaction flux can be obtained from the previous pFBA simulation using the reaction identifier:

In [ ]:
result.fluxes['TPI']

In [ ]:
result.find(['TPI','BIO'])

To retrieve the net conversion equation invoke the `get_net_conversion` method. Exchange reactions with no flux are not presented.

In [ ]:
result.get_net_conversion()




## Retrieving and setting the model objective

The simulation objective, when running FBA or pFBA phenotype simulations, is, by default, the model objective which can be seen using the simulator.

In [ ]:
simul.objective

The simulator may also be used to change the model objective, for example, to optimize the ATP maintenance requirement (ATPM):

In [ ]:
simul.objective = 'ATPM'

In [ ]:
simul.objective

In [ ]:
simul.objective='BIOMASS_Ecoli_core_w_GAM'

## Adding additional constraints to phenotype simulations

Simulations may include additional metabolic constraints on reaction fluxes. From the previous pFBA simulation one can observe that the organism does not produce succinate:

In [ ]:
result.fluxes['EX_succ_e']

Additional constraints may be added to the model so that the organism start to produce this aromatic amino acid. We may alter, for example, the *phosphogluconate dehydrogenase* reaction bounds, among others, starting by verifying its initial bounds:

In [ ]:
simul.get_reaction_bounds('GND')

In [ ]:
constraints = {'GND': 0, # deletion
               'PYK': 0, # deletion
               'ME2': 0, # deletion
              }

# run a pFBA simulation accounting with the new constraint
result = simul.simulate(method='pFBA',constraints=constraints)

result.fluxes['EX_succ_e']

Note that the modifications are not persistently applied to the model, they only exist during the simulation.

We also need to verify that the organism continues to grow:

In [ ]:
result.fluxes['BIOMASS_Ecoli_core_w_GAM']

We can also plot the production envelope:

In [ ]:
%matplotlib inline
from mewpy.visualization.envelope import plot_flux_envelope

plot_flux_envelope(simul,'BIOMASS_Ecoli_core_w_GAM','EX_succ_e',constraints = constraints)

The `simulate` method includes additional parameters, such as the optimization direction. For a full description please refer to the module documentation. 

## Flux Variability Analysis

The simulator interface also allows to perform Flux Variability Analysis (FVA) for all reactions, a list of reactions, or only for succinate:


In [ ]:
# returns a dictionary
simul.FVA(format='df')

In [ ]:
# or a data frame
simul.FVA('EX_succ_e',format='df')

By default, MEWpy sets the model objective fraction to 90%, however this fraction may be altered. For example, one might want to consider a fraction of 10% from optimal growth:

In [ ]:
simul.FVA('EX_succ_e',obj_frac=0.9,format='df')

The FVA simulations are run considering the defined environmental conditions. Additional constraints may be added, or changed, such as the previouly used to increase L-tyrosine production.

In [ ]:
simul.FVA('EX_succ_e', constraints=constraints,format='df')

COBRApy users may have noticed that this same task would have required many additional coding lines if using the COBRApy API directly.

## Genes and reactions essentiality

Gene and reaction essentiality tests identify, respectively, the list of genes and reactions whose deletion would prevent the organism to grow. 

In [ ]:
simul.essential_reactions()

In [ ]:
simul.essential_genes()

For more options and methods please refer to the MEWpy documentation.